# Homework 7

Teach a NN model to predict the prices of the houses from the `kc_house_data.csv'` dataset. 

* clean the data (NaN, repeated values, correlation...)
***I forgot to mention that the target (here `price`) should not be taken into account when making the correlation matrix***!!
* remove outliers (very large, very small values)
* split into train and test 
* use functions:
  * `get_model(input_shape)`
  * `get_regularized_model(input_shape,weight_decay,drop_ate)`
  * `compile_model()`
* use `metrics=['mape']` when compiling model
* `scores=model.evaluate(X_test,y_test)` should return mean absolute percentage error
* use ModelCheckpoint() callback to save the entire model into `.h5` file
* (you can possibly normalize the data before training the algorithm) check this out

    * "id": Numero de identificacion de la vivienda
    * "Date": Fecha en que se vendió la casa
    * "Price": el precio es el objetivo de predicción
    * "Bedrooms": Número de Dormitorios/Casa
    * "Bathrooms": Número de baños/Casa
    * "Sqft_Living": pies cuadrados de la casa
    * "Sqft_Lot": pies cuadrados del lote
    * "Floors": Total de pisos oniveles) en casa
    * "Waterfront": Casa que tiene vista a la costanera
    * "View": Ha sido visto
    * "Condition": Qué tan buena es la condición ogeneral)
    * "Grade": calificación de vivienda, según el sistema del condado de King
    * "Sqft_above": pies cuadrados de la casa aparte del sótano
    * "Sqft_Basement": pies cuadrados del sótano
    * "Yr_Built": año de construcción
    * "Yr_Renovated": año en que se renovó la casa
    * "Zipcode": código postal
    * "Lat": Coordenada de latitud
    * "Long": Coordenada de longitud
    * "Sqft_Living15": Área de la sala de estar en 2015 oimplica algunas renovaciones)
    * "Sqft_Lot15": área del tamaño del lote en 2015 oimplica algunas renovaciones) 

1. Importar, leer e inspeccionar los datos
  * `df.head()`
  * `df.tail()` 
  * `df.columns()`
  * `df.describe()`
  * `df.info()`
  * `df.loc[mascara]` - selecione filas usando "mascara" (ejemplo de mascara (condicion logica): `df['id']>100`) 
2. Completar los datos y organizar las columnas
  * `df.merge()`
  * `df[df.duplicated()] ` - visualiza filas con valores duplicadas atraves todas columnas
  * `df.drop_duplicates(subset=['nombre_columna'])` - eliminar valor duplicadas en la columna
  * `df.drop(['nombre_columna'], axis=1)`           - eliminar columnas innecesarias o repetidas usando funcion
  * `df['nueva_columna']=df['nueva_columna']/10.764` añadir otros nuevos (utilizando cálculos)
  * `df.rename(columns={'nombre_columna_vieja': 'nombre_columna_nueva'})` cambiar el nombre de las columna
  * `df['nombre_columna'] = df['nombre_columna'].astype(str)` cambiar tipo de valores de la columna
3. Comprobar de los tipos de datos y eliminar de los valores repetidos, y `Nan`
  * `df.info()` 
  * `df.isnull().sum()` - soma de todas valores vacios en cada columna
  * `df.isna().sum()`  - soma de todas valores NaN en cada columna
  * `df['sqft_lot'].isnull()` - mascara logica de todos valores null util combinando con `df[mascara]`
  * `df.dropna()` - elimina valores Nan, parametro `axis=0` significa eliminar todas filas con valor NaN
4. Revisar y comprender los datos
  * `df['nombre_columna'].unique()`      - visualizar valores uniqas de la columna
  * `df['nombre_columna'].value_counts()` - sumar frequencia de valores de la columna (se usa para columnas con valores discretos)
  * `df.groupby(['nombre_columna_1','nombre_columna_1']).mean()`
  * `df['nombre_columna'].nlargest(10)` - visualiza 10 mas grandes valore numericas de la columna
  * `df.info()`
  * `df.describe()`
5. Compruebar la correlación entre variables y eliminar las que tengan una correlación superior a 0,8 
  * `corr()`
6. Buscar y eliminar valores atípicos
  * mirar las estadísticas de `df.describe()` max() min()   
  * `df.groupby`
  * histogramas 
  * IQR y boxplot `seaborn.boxplot()`
7. Comprobar la normalidad de la distribución de continua variable
  * `distplot()`
8. Búscar de linealidad y relación general entre variables
  * `pairplot()`
9. Identificar de variables categóricas
  * `matplotlib.scatter_plot()`
  * `seaborn.scatter_plot()`
10. Extraer otras informaciónes válida, por ejemplo usando series de tiempo (time series )
  * `pandas.to_datetime()`
  * `pandas.DatetimeIndex()`

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #visualisation
import seaborn as sns           #visualisation

In [4]:
import pandas as pd
df=pd.read_csv('kc_house_data.csv',sep=',')

In [5]:
df.head(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650.0,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242.0,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000.0,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000.0,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080.0,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,7237550310,20140512T000000,1225000.0,4,4.50,5420,101930.0,1.0,0,0,...,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,1321400060,20140627T000000,257500.0,3,2.25,1715,6819.0,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,2008000270,20150115T000000,291850.0,3,1.50,1060,9711.0,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
8,2414600126,20150415T000000,229500.0,3,1.00,1780,7470.0,1.0,0,0,...,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
9,3793500160,20150312T000000,323000.0,3,2.50,1890,6560.0,2.0,0,0,...,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


In [3]:
!ls

description.txt  homework_07.ipynb  kc_house_data.csv  zipcode_city.csv
